# Health Metrics - Data Cleaning and Analysis
## Table of Contents
1. [Setup Packages and Config](#setup-packages-and-config)
2. [Import Data](#import-data)
3. [Clean the Data](#clean-the-data)
   - [Flatten Nested Columns](#flatten-nested-columns)
   - [Convert Dates to DateTime](#convert-dates-to-datetime)
4. [Validate and Save Cleaned Data](#validate-and-save-cleaned-data)
5. [Data Analysis](#data-analysis)

# 1. Setup Packages and Config 

In [48]:
import pandas as pd
import numpy as np

# 2. Import Data

In [49]:
# Load the JSON data
data = pd.read_json('data.json')["data"]

# Extract 'workouts' and 'metrics' DataFrames
wdf = pd.DataFrame(data["workouts"])

metrics_list = []
for metric in data["metrics"]:
    df = pd.DataFrame(metric['data'])
    df['metric'] = metric['name']
    df['units'] = metric['units']
    metrics_list.append(df)

# Combine all metrics into a single DataFrame
mdf = pd.concat(metrics_list, ignore_index=True)

# 3. Clean the Data
## 3.1 Flatten Nested Columns

In [50]:
# Flatten the nested columns in the 'workouts' DataFrame
def extract_qty_column(df, column_name):
    if column_name in df.columns:
        df[f'{column_name}_qty'] = df[column_name].apply(lambda x: x['qty'] if isinstance(x, dict) else x)
    else:
        df[f'{column_name}_qty'] = np.nan
    return df

# Extract the qty from all relevant columns
columns_to_extract = ['activeEnergyBurned', 'distance', 'lapLength', 'intensity', 'humidity', 'temperature']
for column_name in columns_to_extract:
    wdf = extract_qty_column(wdf, column_name)

# Drop the original columns
wdf.drop(columns=columns_to_extract, axis=1, inplace=True)

# Rename 'qty' for clarity
mdf.rename(columns={'qty': 'value'}, inplace=True)

## 3.2 Convert Dates to DateTime


In [ ]:
# Convert Dates to DateTime objects
wdf['start'] = pd.to_datetime(wdf['start'], format='%Y-%m-%d %H:%M:%S %z')
wdf['end'] = pd.to_datetime(wdf['end'], format='%Y-%m-%d %H:%M:%S %z')
mdf['date'] = pd.to_datetime(mdf['date'], format='%Y-%m-%d %H:%M:%S %z')

# Count the number of missing values in each column 
print(wdf.isnull().sum())

# 4. Validate and Save Cleaned Data

In [ ]:
# Validate cleaned data
print(wdf.head()) 

# Save the cleaned data
wdf.to_csv('cleaned_workouts.csv', index=False)
mdf.to_csv('cleaned_metrics.csv', index=False)

# 5. Example Data Analysis: Next Steps

In [ ]:
# Group by metric for visualization/analysis
grouped = mdf.groupby('metric')

# Print important statistics for each metric
print(grouped['value'].describe())

In [ ]:
import matplotlib.pyplot as plt

# Plot trends for each metric
for metric, group in mdf.groupby('metric'):
    plt.figure(figsize=(10, 5))
    plt.plot(group['date'], group['value'], marker='o', label=metric)
    plt.title(f"Trend for {metric}")
    plt.xlabel("Date")
    plt.ylabel(f"Value ({group['units'].iloc[0]})")
    plt.legend()
    plt.grid(True)
    plt.show()